## Producer

In [ ]:
import yfinance as yf
import json
from time import sleep
import sys
import time
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# if sys.version_info >= (3, 12, 0):
#     import six
#     sys.modules['kafka.vendor.six.moves'] = six.moves

from confluent_kafka import Producer as KafkaProducer


config = {
    # User-specific properties that you must set
    'bootstrap.servers': os.getenv('BOOTSTRAP_SERVERS'),
    'sasl.username': os.getenv('SASL_USERNAME'),
    'sasl.password': os.getenv('SASL_PASSWORD'),

    # Fixed properties
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'acks': 'all'
}

# Create Producer instance
producer = KafkaProducer(config)

ticker = "ZOMATO.NS"
kafka_topic = "zomato_stock"
counter = 0
while counter<=20:

    zomato = {}

    stock = yf.Ticker(ticker)
    zomato['Price'] = stock.fast_info.last_price
    
    zomato['Name'] = stock.info['shortName']
    zomato['Timestamp'] = time.time()
    print(zomato)

    producer.produce(kafka_topic, json.dumps(zomato))
    sleep(12)
    counter += 1

{'Price': 288.25, 'Name': 'ZOMATO LIMITED', 'Timestamp': 1734162615.2084439}
{'Price': 288.25, 'Name': 'ZOMATO LIMITED', 'Timestamp': 1734162627.3996458}
{'Price': 288.25, 'Name': 'ZOMATO LIMITED', 'Timestamp': 1734162639.595969}


## Consumer

In [1]:
from confluent_kafka import Consumer
from dotenv import load_dotenv
import os

# Kafka consumer configuration
# Load environment variables from .env file
load_dotenv()

config = {
    'bootstrap.servers': os.getenv('BOOTSTRAP_SERVERS'),
    'sasl.username': os.getenv('SASL_USERNAME'),
    'sasl.password': os.getenv('SASL_PASSWORD'),
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'group.id': 'my-consumer-group',
    'auto.offset.reset': 'latest',  # Possible values: 'earliest', 'latest', 'none'
    'enable.auto.commit': 'false'
}

# Create Kafka consumer instance
consumer = Consumer(config)

# Subscribe to Kafka topic
kafka_topic = 'zomato_stock'
consumer.subscribe([kafka_topic])

# Consume messages from Kafka
while True:
    message = consumer.poll(1.0)

    if message is None:
        continue

    if message.error():
        print(f"Error: {message.error()}")
        continue

    # Process the received message
    value = message.value().decode('utf-8')
    print(f"Received message: {value} {type(value)}")

    # Commit the consumed message offset
    consumer.commit(message)

# Close Kafka consumer
consumer.close()

Received message: {"Price": 288.25, "Name": "ZOMATO LIMITED", "Timestamp": 1734162615.2084439} <class 'str'>
Received message: {"Price": 288.25, "Name": "ZOMATO LIMITED", "Timestamp": 1734162627.3996458} <class 'str'>
Received message: {"Price": 288.25, "Name": "ZOMATO LIMITED", "Timestamp": 1734162639.595969} <class 'str'>


KeyboardInterrupt: 

# when market closed

In [1]:
import yfinance as yf
stock = yf.Ticker('ZOMATO.NS')

from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()

# if sys.version_info >= (3, 12, 0):
#     import six
#     sys.modules['kafka.vendor.six.moves'] = six.moves

from confluent_kafka import Producer as KafkaProducer


config = {
    # User-specific properties that you must set
    'bootstrap.servers': os.getenv('BOOTSTRAP_SERVERS'),
    'sasl.username': os.getenv('SASL_USERNAME'),
    'sasl.password': os.getenv('SASL_PASSWORD'),

    # Fixed properties
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'acks': 'all'
}

# Create Producer instance
producer = KafkaProducer(config)

In [2]:
import json, time
import pandas as pd
for record in json.loads(stock.history(period="1mo").reset_index().to_json(orient='records')):
    date = record['Date']
    price = record['Close']
    print(f"Timestamp: {date}, {pd.to_datetime(date, unit='ms')},Price: {price}")

    producer.produce('zomato_stock', json.dumps({'Timestamp': date, 'Price': price}))
    producer.flush()
    time.sleep(5)
producer.flush()

Timestamp: 1731436200000, 2024-11-12 18:30:00,Price: 258.3999938965
Timestamp: 1731522600000, 2024-11-13 18:30:00,Price: 269.6600036621
Timestamp: 1731868200000, 2024-11-17 18:30:00,Price: 270.7900085449
Timestamp: 1731954600000, 2024-11-18 18:30:00,Price: 271.3599853516
Timestamp: 1732127400000, 2024-11-20 18:30:00,Price: 266.8099975586
Timestamp: 1732213800000, 2024-11-21 18:30:00,Price: 264.200012207
Timestamp: 1732473000000, 2024-11-24 18:30:00,Price: 273.3099975586
Timestamp: 1732559400000, 2024-11-25 18:30:00,Price: 280.1099853516
Timestamp: 1732645800000, 2024-11-26 18:30:00,Price: 284.7200012207
Timestamp: 1732732200000, 2024-11-27 18:30:00,Price: 286.1300048828
Timestamp: 1732818600000, 2024-11-28 18:30:00,Price: 279.7600097656
Timestamp: 1733077800000, 2024-12-01 18:30:00,Price: 282.5
Timestamp: 1733164200000, 2024-12-02 18:30:00,Price: 279.8500061035
Timestamp: 1733250600000, 2024-12-03 18:30:00,Price: 286.25
Timestamp: 1733337000000, 2024-12-04 18:30:00,Price: 299.350006103

0

In [19]:
stock.history(period="5d")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-12-09 00:00:00+05:30,304.700012,304.700012,291.799988,295.299988,51131895,0.0,0.0
2024-12-10 00:00:00+05:30,296.750000,299.450012,295.000000,295.850006,31333039,0.0,0.0
2024-12-11 00:00:00+05:30,290.000000,294.299988,288.600006,291.799988,39053928,0.0,0.0
2024-12-12 00:00:00+05:30,291.799988,295.000000,284.100006,284.899994,38405304,0.0,0.0
2024-12-13 00:00:00+05:30,279.000000,288.950012,278.200012,288.250000,47279406,0.0,0.0


## zomato volume analysis

In [1]:
import yfinance as yf
stock = yf.Ticker('ZOMATO.NS')

In [24]:
df = stock.history(period="1y", interval="1mo").reset_index()
df['Date'] = df['Date'].dt.strftime('%Y-%m')
df.groupby('Date')['Volume'].sum().reset_index()

,Date,Volume
0,2024-01,1131629423
1,2024-02,1802870225
2,2024-03,891100116
3,2024-04,718774624
4,2024-05,1043386574
5,2024-06,874273702
6,2024-07,882355583
7,2024-08,1770805158
8,2024-09,1325696904
9,2024-10,1198135103


In [27]:
stock.major_holders

Breakdown,Value
insidersPercentHeld,0.22488
institutionsPercentHeld,0.38948
institutionsFloatPercentHeld,0.50248
institutionsCount,324.00000


In [31]:
stock.info['industry']

'Internet Retail'

In [37]:
stock.info['bookValue']

24.225

In [38]:
stock.balance_sheet

,2024-03-31,2023-03-31,2022-03-31,2021-03-31,2020-03-31
Treasury Shares Number,139528706.0,189534357.0,228992198.0,NaN,NaN
Ordinary Shares Number,8680255038.0,8363975413.0,7642940578.0,7845179676.0,NaN
Share Issued,8819783744.0,8553509770.0,7871932776.0,7845179676.0,NaN
Total Debt,7490000000.0,5070000000.0,703000000.0,725000000.0,NaN
Tangible Book Value,149430000000.0,137530000000.0,152163000000.0,66434000000.0,NaN
...,...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,34050000000.0,94080000000.0,65974000000.0,37053000000.0,NaN
Other Short Term Investments,30960000000.0,91900000000.0,62051000000.0,33988000000.0,NaN
Cash And Cash Equivalents,3090000000.0,2180000000.0,3923000000.0,3065000000.0,NaN
Cash Equivalents,390000000.0,50000000.0,245000000.0,161000000.0,NaN


In [40]:
# get which mutual funds hold the stock and invested the most
stock.institutional_holders

""


In [5]:
import pandas as pd
news_df = pd.DataFrame(stock.news)
news_df

,uuid,title,publisher,link,providerPublishTime,type,thumbnail,relatedTickers
0,5f079167-4e5b-3012-a5d2-e2b496bb34d9,FRN Variable Rate Fix,Business Wire,https://finance.yahoo.com/news/frn-variable-ra...,1734111120,STORY,NaN,NaN
1,bbe193af-6e4a-3bd4-9aed-d562aa49dcff,The 'five freedoms' this veteran entrepreneur ...,Yahoo Finance Video,https://finance.yahoo.com/video/five-freedoms-...,1734111047,VIDEO,{'resolutions': [{'url': 'https://s.yimg.com/u...,NaN
2,fd834a6f-3984-4be4-a2c3-0276a43a0b60,"Anker's 3-in-1 power bank drops to just $20, p...",Engadget,https://finance.yahoo.com/m/fd834a6f-3984-4be4...,1734111041,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,NaN
3,347320c7-dc90-3655-ac96-ff07e63e856a,EPH European Property Holdings PLC announces c...,GlobeNewswire,https://finance.yahoo.com/news/eph-european-pr...,1734111000,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,[EPH.SW]
4,91aad472-65a2-3246-9185-45fa3fe07ff7,Global stocks pull lower on US rate concerns,AFP,https://finance.yahoo.com/news/asian-markets-r...,1734110860,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,NaN
5,e9ac8003-4311-3d67-84a9-b813f67023f1,Market Chatter: Sycamore Partners Approaches B...,MT Newswires,https://finance.yahoo.com/news/market-chatter-...,1734110854,STORY,NaN,[WBA]
6,54c3c542-ac57-3d31-ab5f-06cebba1051a,Eldorado Gold Maintained at Outperform at BMO ...,MT Newswires,https://finance.yahoo.com/news/eldorado-gold-m...,1734110830,STORY,NaN,[EGO]
7,2324da26-0f86-3850-8352-f3cb0157de3b,"Josh Kesselman, Founder of RAW Rolling Papers,...",Newsfile,https://finance.yahoo.com/news/josh-kesselman-...,1734110760,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,NaN


In [9]:
import datetime
news_df['providerPublishTime'].map(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

0    2024-12-13 23:02:00
1    2024-12-13 23:00:47
2    2024-12-13 23:00:41
3    2024-12-13 23:00:00
4    2024-12-13 22:57:40
5    2024-12-13 22:57:34
6    2024-12-13 22:57:10
7    2024-12-13 22:56:00
Name: providerPublishTime, dtype: object

In [10]:
stock.recommendations

,period,strongBuy,buy,hold,sell,strongSell
0,0m,5,18,0,3,0
1,-1m,5,18,0,3,0
2,-2m,5,18,0,3,0
3,-3m,5,18,0,3,0
